In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
train = pd.read_csv('C:/Users/dudwo/Desktop/ML/Data/titanic/train.csv')
test = pd.read_csv('C:/Users/dudwo/Desktop/ML/Data/titanic/test.csv')
train = train.drop(['Ticket','Cabin'],axis = 1) 
test = test.drop(['Ticket','Cabin'],axis = 1)   
train = train.fillna({"Embarked" : "S"})
embarked_mapping = {"S":1,"C":2,"Q":3}
train["Embarked"] = train["Embarked"].map(embarked_mapping)
test["Embarked"] = test["Embarked"].map(embarked_mapping)
combine = [train, test]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.',expand= False)
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(
        ['Lady', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    dataset['Title'] = dataset['Title'].replace('Mile', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr":1,"Miss":2,"Mrs":3,"Master":4,"Royal":5,"Rare":6}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
train = train.drop(['Name','PassengerId'],axis =1)
test = test.drop(['Name','PassengerId'],axis =1)
combine = [train,test]
sex_mapping = {"male":0,"female":1}
for dataset in combine :
    dataset['Sex'] = dataset["Sex"].map(sex_mapping)
train['Age'] = train['Age'].fillna(-0.5)
test['Age'] = test['Age'].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown','Baby','Child','Teenager','Student','Young Adult','Adult','Senior']
train['AgeGroup'] = pd.cut(train["Age"],bins, labels= labels)
test['AgeGroup'] = pd.cut(test["Age"],bins, labels=labels)
age_title_mapping = {1:"Young Adult",2:"Student",3:"Adult",4:"Baby",5:"Adult",6:"Adult",}
for x in range(len(train["AgeGroup"])):
    if train["AgeGroup"][x] == "Unknown" :
        train["AgeGroup"][x] = age_title_mapping[train["Title"][x]]
for x in range(len(test["AgeGroup"])):
    if test["AgeGroup"][x] == "Unknown" :
        test["AgeGroup"][x] = age_title_mapping[test["Title"][x]]
age_mappaing = {"Baby":1,"Child":2,"Teenager":3,"Student":4,"Young Adult":5,"Adult":6,"Senior":7}
train['AgeGroup'] = train['AgeGroup'].map(age_mappaing)
test['AgeGroup'] = test['AgeGroup'].map(age_mappaing)
train = train.drop(['Age'],axis =1)
test = test.drop(['Age'],axis =1)
train['FareBand'] = pd.qcut(train['Fare'], 4, labels= [1,2,3,4])
test['FareBand'] = pd.qcut(train['Fare'],4, labels=[1,2,3,4])
train = train.drop(['Fare'],axis = 1)
test = test.drop(['Fare'],axis=1)

In [12]:
train_data = train.drop('Survived', axis = 1)
target = train['Survived']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    891 non-null int64
Title       891 non-null float64
AgeGroup    891 non-null int64
FareBand    891 non-null category
dtypes: category(1), float64(1), int64(6)
memory usage: 49.9 KB


In [13]:
# Cross_Validation 실행

from sklearn.model_selection import KFold
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [30]:
# RandomForestClassifier 실행

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=13)
clf.fit(train_data,target)
print(clf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=13, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [31]:
# Cross_val_score 점수
# F1 Scoring

from sklearn.model_selection import cross_val_score
scoring = 'f1'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)
print(round(np.mean(score)*100,2))

[0.77333333 0.73684211 0.70967742 0.71428571 0.8        0.73333333
 0.79452055 0.78688525 0.76056338 0.73684211]
75.46


In [32]:
# Accuracy Scoring

from sklearn.model_selection import cross_val_score
scoring = 'accuracy'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)
print(round(np.mean(score)*100,2))

[0.81111111 0.83146067 0.80898876 0.78651685 0.85393258 0.84269663
 0.83146067 0.83146067 0.80898876 0.78651685]
81.93


In [33]:
# Recall Scoring

from sklearn.model_selection import cross_val_score
scoring = 'recall'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)
print(round(np.mean(score)*100,2))

[0.82051282 0.73333333 0.67741935 0.69444444 0.7037037  0.70967742
 0.80555556 0.77419355 0.74358974 0.66666667]
73.29


In [34]:
# Precision Scoring

from sklearn.model_selection import cross_val_score
scoring = 'precision'
score = cross_val_score(clf,train_data,target,cv=k_fold,n_jobs=1,scoring=scoring)
print(score)
print(round(np.mean(score)*100,2))

[0.74418605 0.78571429 0.6875     0.80645161 0.74074074 0.75
 0.78378378 0.80769231 0.81818182 0.88235294]
78.07


In [36]:
# test 파일 예측 실행

# test 파일 PassengerId 열 불러오기 위함
test2 = pd.read_csv('C:/Users/dudwo/Desktop/ML/Data/titanic/test.csv')

# 예측
prediction = clf.predict(test)
submission = pd.DataFrame({
    "PassengerId" : test2["PassengerId"],
    "Survived" : prediction
})

print(submission)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         1
4            896         0
5            897         0
6            898         1
7            899         0
8            900         1
9            901         0
10           902         0
11           903         1
12           904         1
13           905         0
14           906         1
15           907         1
16           908         0
17           909         0
18           910         0
19           911         1
20           912         0
21           913         1
22           914         1
23           915         1
24           916         1
25           917         0
26           918         1
27           919         0
28           920         0
29           921         0
..           ...       ...
388         1280         0
389         1281         0
390         1282         0
391         1283         1
392         1284         1
3